In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.firefox.options import Options
from mendeleev import element
import pandas as pd

def ALL(ion,cenwave):
    """
    Inputs:
    ion = Ion of interest, e.g. 'OVI'
    cenwave = Approximate wavelength of the transition
    
    
    Returns tuple consisting of central wavelength, oscillator strength, gamma and the atomic weight. The information is obtained from 
    Peter van Hoof's website: https://www.pa.uky.edu/~peter/newpage/
    
    Usage example:
    ALL('OVI',1031.5)
    
    Gives:
    (1031.93, 0.133, 416000000.0, 15.999)
    """
    
    romannum=['I','II','III','IV','V','VI','VII','VIII','IX','X']
    if ion[1:] not in romannum:
        ionname=ion[0:2]
        trans=ion[2:]
    else:
        ionname=ion[0:1]
        trans=ion[1:]
        

    options = Options()
    options.headless = True
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox(options=options)

    # go to the atom list data page
    driver.get("https://www.pa.uky.edu/~peter/newpage/")

    # set wavelength range
    wavr=driver.find_element_by_name('wavl')
    wavr.send_keys('{}+/-1'.format(cenwave))

    
    #select the wavelength unit 

    select=Select(driver.find_element_by_name('wave'))
    select.select_by_visible_text('Angstrom')

    #select the type of wavelength unit
    select=Select(driver.find_element_by_name('air'))
    select.select_by_visible_text('Vacuum')

    ion=driver.find_element_by_name('elmion')

    # send the ion of interest into the field

    ion.send_keys('{} {}'.format(ionname,trans))

    # exclude information without atomic data
    driver.find_element_by_xpath("//input[@value='excl']").click()

    # # choose all types of ions
    # driver.find_element_by_xpath("//input[@value='All']").click()

    #
    driver.find_element_by_xpath("//input[@value='wacc']").click()
    driver.find_element_by_xpath("//input[@value='spec']").click()
    driver.find_element_by_xpath("//input[@value='type']").click()
    driver.find_element_by_xpath("//input[@value='term']").click()
    driver.find_element_by_xpath("//input[@value='angm']").click()

    #check the tick-boxes
    driver.find_element_by_xpath("//input[@value='prob']").click()
    # driver.find_element_by_xpath("//input[@value='as_a']").click()
    driver.find_element_by_xpath("//input[@value='as_f']").click()

    driver.find_element_by_xpath("//input[@value='ener']").click()

    #driver.find_element_by_xpath("//input[@value='HTML ']").click()
    driver.find_element_by_xpath("//input[@value='Plain']").click()


    #select the 5000 output
    select=Select(driver.find_element_by_name('mlin'))
    select.select_by_visible_text('5000')


    #send query
    driver.find_element_by_xpath("//input[@value='    Send Query   ']").click()

    soup = BeautifulSoup(driver.page_source)
    if soup.text.split('\n')[-2].split(' ')[1] == 'no' or soup.text.split('\n')[1].split(' ')[2] == 'not':
        print ("{}_{} not found".format(ionname,trans))
    
    else:
        cols = ['WAVE', 'ERR-WAVE','GAMMA','FAC']
        dat = pd.DataFrame(columns = cols)
        for i in soup.text.split('\n')[19:-1]:
            info=[j for j in i.split(' ') if j!='']
            dat = dat.append({'WAVE': float(info[0]), 'ERR-WAVE':float(info[1]),'GAMMA':float(info[2]),'FAC':float(info[3])},ignore_index=True)    

        idx = dat['WAVE'].sub(cenwave).abs().idxmin()
       
        return dat.loc[[idx]]['WAVE'].item(), dat.loc[[idx]]['FAC'].item(), dat.loc[[idx]]['GAMMA'].item(), element(ionname).atomic_weight
        